## Extraer Nombres

In [1]:
from pathlib import Path

# === Ruta base donde buscar ===
carpeta_base = Path("20251003/20251003")

# === Buscar todos los archivos cuyo nombre empiece por 'ADP_DTM_' ===
archivos_encontrados = list(carpeta_base.glob("ADP_DTM_*"))

# === Mostrar resultados ===
if archivos_encontrados:
    print("[DEBUG] Archivos encontrados:")
    for archivo in archivos_encontrados:
        print("-", archivo.name)
else:
    print("[DEBUG] No se encontraron archivos que empiecen por 'ADP_DTM_'.")


[DEBUG] Archivos encontrados:
- ADP_DTM_DIM.Actividades.csv
- ADP_DTM_DIM.Bodega.csv
- ADP_DTM_DIM.CapituloPresupuesto.csv
- ADP_DTM_DIM.ControlClaseOrigen.csv
- ADP_DTM_DIM.Empresa.csv
- ADP_DTM_DIM.EspecicficacionDePedidos.csv
- ADP_DTM_DIM.EspecificacionDeActas.csv
- ADP_DTM_DIM.EspecificacionDeContratos.csv
- ADP_DTM_DIM.EspecificacionDeEntradasAlmacen.csv
- ADP_DTM_DIM.EspecificacionEjecucionCliente.csv
- ADP_DTM_DIM.EstadoEnvioDocumento.csv
- ADP_DTM_DIM.EstadoPorDocumento.csv
- ADP_DTM_DIM.Fecha.csv
- ADP_DTM_DIM.Insumo.csv
- ADP_DTM_DIM.Items.csv
- ADP_DTM_DIM.NivelesPresupuesto.csv
- ADP_DTM_DIM.OrigenDelDocumento.csv
- ADP_DTM_DIM.Proyecto.csv
- ADP_DTM_DIM.Tercero.csv
- ADP_DTM_DIM.TipoContrato.csv
- ADP_DTM_DIM.TipoDeDescuento.csv
- ADP_DTM_DIM.TiposPolizas.csv
- ADP_DTM_DIM.Usuario.csv
- ADP_DTM_DIM.VariablesAdicionalesContratos.csv
- ADP_DTM_DIM.VariablesAdicionalesProyecto.csv
- ADP_DTM_DIM.ZonasProyecto.csv
- ADP_DTM_FACT.Acta.csv
- ADP_DTM_FACT.ActasDescuentos.csv
- AD

In [5]:
import pandas as pd
from pathlib import Path

# === Ruta base donde buscar ===
carpeta_base = Path("20251003/20251003")

# === Buscar todos los archivos que empiecen por 'ADP_DTM_' ===
archivos_csv = list(carpeta_base.glob("ADP_DTM_*.csv"))

print(f"[DEBUG] {len(archivos_csv)} archivos encontrados.")

# === Función para detectar variables de fecha ===
def detectar_columnas_fecha(nombre_archivo: Path) -> list[str]:
    try:
        # Leer solo la primera fila (encabezados)
        encabezados = pd.read_csv(nombre_archivo, nrows=0).columns
        # Detectar columnas con palabras clave relacionadas a fecha
        columnas_fecha = [
            col for col in encabezados
            if any(palabra in col.lower() for palabra in ["fecha", "date", "fch"])
        ]
        print(f"[DEBUG] {nombre_archivo.name}: {columnas_fecha}")
        return columnas_fecha
    except Exception as e:
        print(f"[DEBUG] Error leyendo {nombre_archivo.name}: {e}")
        return []

# === Recorrer y registrar resultados ===
resultado = {}
for archivo in archivos_csv:
    columnas = detectar_columnas_fecha(archivo)
    if columnas:
        resultado[archivo.name] = columnas

# === Mostrar resumen ===
print("\n=== Resumen de columnas tipo fecha ===")
for nombre, columnas in resultado.items():
    print(f"{nombre}: {', '.join(columnas)}")


[DEBUG] 50 archivos encontrados.
[DEBUG] ADP_DTM_DIM.Actividades.csv: []
[DEBUG] ADP_DTM_DIM.Bodega.csv: []
[DEBUG] ADP_DTM_DIM.CapituloPresupuesto.csv: []
[DEBUG] ADP_DTM_DIM.ControlClaseOrigen.csv: []
[DEBUG] ADP_DTM_DIM.Empresa.csv: []
[DEBUG] ADP_DTM_DIM.EspecicficacionDePedidos.csv: []
[DEBUG] ADP_DTM_DIM.EspecificacionDeActas.csv: []
[DEBUG] ADP_DTM_DIM.EspecificacionDeContratos.csv: ['Fecha de creacion', 'Fecha Inicio', 'Fecha Fin']
[DEBUG] ADP_DTM_DIM.EspecificacionDeEntradasAlmacen.csv: []
[DEBUG] ADP_DTM_DIM.EspecificacionEjecucionCliente.csv: []
[DEBUG] ADP_DTM_DIM.EstadoEnvioDocumento.csv: []
[DEBUG] ADP_DTM_DIM.EstadoPorDocumento.csv: []
[DEBUG] ADP_DTM_DIM.Fecha.csv: ['SkIdFecha', 'Fecha']
[DEBUG] ADP_DTM_DIM.Insumo.csv: ['Fecha Creacion', 'Fecha Modificacion']
[DEBUG] ADP_DTM_DIM.Items.csv: []
[DEBUG] ADP_DTM_DIM.NivelesPresupuesto.csv: []
[DEBUG] ADP_DTM_DIM.OrigenDelDocumento.csv: []
[DEBUG] ADP_DTM_DIM.Proyecto.csv: ['Fecha De Elaboracion', 'Fecha De Inicio', 'Fecha D

In [6]:
import pandas as pd
from pathlib import Path

# === Ruta base ===
carpeta_base = Path("20251003/20251003")
archivos_csv = list(carpeta_base.glob("ADP_DTM_*.csv"))

print(f"[DEBUG] {len(archivos_csv)} archivos encontrados.")

def detectar_columnas_fecha(nombre_archivo: Path) -> list[str]:
    """Detecta columnas de fecha según el encabezado."""
    try:
        encabezados = pd.read_csv(nombre_archivo, nrows=0).columns
        columnas_fecha = [
            col for col in encabezados
            if any(p in col.lower() for p in ["fecha", "date", "fch"])
        ]
        return columnas_fecha
    except Exception as e:
        print(f"[DEBUG] Error leyendo encabezados de {nombre_archivo.name}: {e}")
        return []

def contar_por_año(nombre_archivo: Path, columnas_fecha: list[str]) -> pd.DataFrame:
    """Cuenta la frecuencia de registros por año en las columnas de fecha detectadas."""
    try:
        df = pd.read_csv(nombre_archivo, usecols=columnas_fecha, low_memory=False)
        tabla_final = []
        for col in columnas_fecha:
            # Convertir a fecha
            df[col] = pd.to_datetime(df[col], errors="coerce")
            # Contar por año
            conteo = df[col].dt.year.value_counts().sort_index()
            temp = pd.DataFrame({
                "archivo": nombre_archivo.name,
                "columna": col,
                "año": conteo.index,
                "frecuencia": conteo.values
            })
            tabla_final.append(temp)
        return pd.concat(tabla_final, ignore_index=True)
    except Exception as e:
        print(f"[DEBUG] Error procesando {nombre_archivo.name}: {e}")
        return pd.DataFrame(columns=["archivo", "columna", "año", "frecuencia"])

# === Procesar todos los archivos ===
todas_las_tablas = []
for archivo in archivos_csv:
    columnas_fecha = detectar_columnas_fecha(archivo)
    if columnas_fecha:
        print(f"[DEBUG] {archivo.name}: columnas de fecha -> {columnas_fecha}")
        tabla = contar_por_año(archivo, columnas_fecha)
        if not tabla.empty:
            todas_las_tablas.append(tabla)

# === Unir resultados ===
if todas_las_tablas:
    resumen = pd.concat(todas_las_tablas, ignore_index=True)
    print("\n=== Tabla de frecuencias por año ===")
    print(resumen)
    # Guardar opcionalmente
    resumen.to_csv("frecuencias_fechas_ADP_DTM.csv", index=False)
    print("\n[DEBUG] Guardado como frecuencias_fechas_ADP_DTM.csv")
else:
    print("[DEBUG] No se generó ninguna tabla de frecuencias.")


[DEBUG] 50 archivos encontrados.
[DEBUG] ADP_DTM_DIM.EspecificacionDeContratos.csv: columnas de fecha -> ['Fecha de creacion', 'Fecha Inicio', 'Fecha Fin']
[DEBUG] ADP_DTM_DIM.Fecha.csv: columnas de fecha -> ['SkIdFecha', 'Fecha']
[DEBUG] ADP_DTM_DIM.Insumo.csv: columnas de fecha -> ['Fecha Creacion', 'Fecha Modificacion']
[DEBUG] ADP_DTM_DIM.Proyecto.csv: columnas de fecha -> ['Fecha De Elaboracion', 'Fecha De Inicio', 'Fecha De Finalizacion']


C:\Users\aleja\AppData\Local\Temp\ipykernel_20760\946904950.py:30: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df[col] = pd.to_datetime(df[col], errors="coerce")
C:\Users\aleja\AppData\Local\Temp\ipykernel_20760\946904950.py:30: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df[col] = pd.to_datetime(df[col], errors="coerce")
C:\Users\aleja\AppData\Local\Temp\ipykernel_20760\946904950.py:30: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df[col] = pd.to_datetime(df[col], errors="coerce")
C:\Users\aleja\AppData\Local\Temp\ipykernel_20760\946904950.py:30: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specif

[DEBUG] ADP_DTM_FACT.Acta.csv: columnas de fecha -> ['SkIdFecha']
[DEBUG] ADP_DTM_FACT.ActasDescuentos.csv: columnas de fecha -> ['SkIdFecha']
[DEBUG] ADP_DTM_FACT.Anticipo.csv: columnas de fecha -> ['SkIdFechaAnticipo', 'SkIdFechaPago']
[DEBUG] ADP_DTM_FACT.AuditoriaActas.csv: columnas de fecha -> ['SkIdFecha']
[DEBUG] ADP_DTM_FACT.AuditoriaContratos.csv: columnas de fecha -> ['SkIdFecha']
[DEBUG] ADP_DTM_FACT.AuditoriaEntradasAlmacen.csv: columnas de fecha -> ['SkIdFecha']
[DEBUG] ADP_DTM_FACT.AuditoriaPedidos.csv: columnas de fecha -> ['SkIdFecha']
[DEBUG] ADP_DTM_FACT.Compras.csv: columnas de fecha -> ['SkIdFechaCompra', 'SkIdFechaEntrega', 'SkIdFechaCierre']
[DEBUG] ADP_DTM_FACT.ContratosPolizas.csv: columnas de fecha -> ['SkIdFechaVigenciaDesde', 'SkIdFechaVigenciaHasta']
[DEBUG] ADP_DTM_FACT.ControlProyecto.csv: columnas de fecha -> ['SkIdFecha']
[DEBUG] ADP_DTM_FACT.Devoluciones.csv: columnas de fecha -> ['SkIdFecha']
[DEBUG] ADP_DTM_FACT.EjecucionCliente.csv: columnas de fecha

In [7]:
import pandas as pd
from pathlib import Path

# === Ruta base ===
carpeta_base = Path("20251003/20251003")
archivos_csv = list(carpeta_base.glob("ADP_DTM_*.csv"))

print(f"[DEBUG] {len(archivos_csv)} archivos encontrados.")

def detectar_columnas_fecha(nombre_archivo: Path) -> list[str]:
    """Detecta columnas de fecha según el encabezado."""
    try:
        encabezados = pd.read_csv(nombre_archivo, nrows=0).columns
        columnas_fecha = [
            col for col in encabezados
            if any(p in col.lower() for p in ["fecha", "date", "fch"])
        ]
        return columnas_fecha
    except Exception as e:
        print(f"[DEBUG] Error leyendo encabezados de {nombre_archivo.name}: {e}")
        return []

def contar_por_proyecto(nombre_archivo: Path, columnas_fecha: list[str]) -> pd.DataFrame:
    """Cuenta la frecuencia de registros por SkIdProyecto y columnas de fecha."""
    try:
        # Verificar si la columna SkIdProyecto existe
        encabezados = pd.read_csv(nombre_archivo, nrows=0).columns
        if "SkIdProyecto" not in encabezados:
            print(f"[DEBUG] {nombre_archivo.name}: sin columna 'SkIdProyecto', omitido.")
            return pd.DataFrame(columns=["archivo", "columna", "SkIdProyecto", "frecuencia"])
        
        columnas_a_leer = ["SkIdProyecto"] + columnas_fecha
        df = pd.read_csv(nombre_archivo, usecols=columnas_a_leer, low_memory=False)
        tabla_final = []
        
        for col in columnas_fecha:
            df[col] = pd.to_datetime(df[col], errors="coerce")
            conteo = df.groupby("SkIdProyecto")[col].count().reset_index()
            conteo.rename(columns={col: "frecuencia"}, inplace=True)
            conteo["archivo"] = nombre_archivo.name
            conteo["columna"] = col
            tabla_final.append(conteo)

        return pd.concat(tabla_final, ignore_index=True)
    except Exception as e:
        print(f"[DEBUG] Error procesando {nombre_archivo.name}: {e}")
        return pd.DataFrame(columns=["archivo", "columna", "SkIdProyecto", "frecuencia"])

# === Procesar todos los archivos ===
todas_las_tablas = []
for archivo in archivos_csv:
    columnas_fecha = detectar_columnas_fecha(archivo)
    if columnas_fecha:
        print(f"[DEBUG] {archivo.name}: columnas de fecha -> {columnas_fecha}")
        tabla = contar_por_proyecto(archivo, columnas_fecha)
        if not tabla.empty:
            todas_las_tablas.append(tabla)

# === Unir resultados ===
if todas_las_tablas:
    resumen = pd.concat(todas_las_tablas, ignore_index=True)
    print("\n=== Tabla de frecuencias por SkIdProyecto ===")
    print(resumen)
    resumen.to_csv("frecuencias_por_SkIdProyecto_ADP_DTM.csv", index=False)
    print("\n[DEBUG] Guardado como frecuencias_por_SkIdProyecto_ADP_DTM.csv")
else:
    print("[DEBUG] No se generó ninguna tabla de frecuencias.")


[DEBUG] 50 archivos encontrados.
[DEBUG] ADP_DTM_DIM.EspecificacionDeContratos.csv: columnas de fecha -> ['Fecha de creacion', 'Fecha Inicio', 'Fecha Fin']
[DEBUG] ADP_DTM_DIM.EspecificacionDeContratos.csv: sin columna 'SkIdProyecto', omitido.
[DEBUG] ADP_DTM_DIM.Fecha.csv: columnas de fecha -> ['SkIdFecha', 'Fecha']
[DEBUG] ADP_DTM_DIM.Fecha.csv: sin columna 'SkIdProyecto', omitido.
[DEBUG] ADP_DTM_DIM.Insumo.csv: columnas de fecha -> ['Fecha Creacion', 'Fecha Modificacion']
[DEBUG] ADP_DTM_DIM.Insumo.csv: sin columna 'SkIdProyecto', omitido.
[DEBUG] ADP_DTM_DIM.Proyecto.csv: columnas de fecha -> ['Fecha De Elaboracion', 'Fecha De Inicio', 'Fecha De Finalizacion']
[DEBUG] ADP_DTM_FACT.Acta.csv: columnas de fecha -> ['SkIdFecha']


C:\Users\aleja\AppData\Local\Temp\ipykernel_20760\479964316.py:37: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df[col] = pd.to_datetime(df[col], errors="coerce")
C:\Users\aleja\AppData\Local\Temp\ipykernel_20760\479964316.py:37: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df[col] = pd.to_datetime(df[col], errors="coerce")
C:\Users\aleja\AppData\Local\Temp\ipykernel_20760\479964316.py:37: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df[col] = pd.to_datetime(df[col], errors="coerce")


[DEBUG] ADP_DTM_FACT.ActasDescuentos.csv: columnas de fecha -> ['SkIdFecha']
[DEBUG] ADP_DTM_FACT.Anticipo.csv: columnas de fecha -> ['SkIdFechaAnticipo', 'SkIdFechaPago']
[DEBUG] ADP_DTM_FACT.AuditoriaActas.csv: columnas de fecha -> ['SkIdFecha']
[DEBUG] ADP_DTM_FACT.AuditoriaContratos.csv: columnas de fecha -> ['SkIdFecha']
[DEBUG] ADP_DTM_FACT.AuditoriaEntradasAlmacen.csv: columnas de fecha -> ['SkIdFecha']
[DEBUG] ADP_DTM_FACT.AuditoriaPedidos.csv: columnas de fecha -> ['SkIdFecha']
[DEBUG] ADP_DTM_FACT.Compras.csv: columnas de fecha -> ['SkIdFechaCompra', 'SkIdFechaEntrega', 'SkIdFechaCierre']
[DEBUG] ADP_DTM_FACT.ContratosPolizas.csv: columnas de fecha -> ['SkIdFechaVigenciaDesde', 'SkIdFechaVigenciaHasta']
[DEBUG] ADP_DTM_FACT.ControlProyecto.csv: columnas de fecha -> ['SkIdFecha']
[DEBUG] ADP_DTM_FACT.Devoluciones.csv: columnas de fecha -> ['SkIdFecha']
[DEBUG] ADP_DTM_FACT.EjecucionCliente.csv: columnas de fecha -> ['SkIdFecha']
[DEBUG] ADP_DTM_FACT.EjecucionEstandar.csv: colu